In [26]:
from google.colab import files
import zipfile
import os

# Upload ZIP file
uploaded = files.upload()  # Browse and select the ZIP file

# Extract the uploaded zip file (assuming it's named 'animal_data.zip')
with zipfile.ZipFile("animal_data.zip", 'r') as zip_ref:
    zip_ref.extractall("animal_data_file")  # Extracts to a folder in current directory

print("Extraction complete!")


Saving animal_data.zip to animal_data (2).zip
Extraction complete!


In [30]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
import keras_tuner as kt
import os


**1. Problem Statement**

In [ ]:
# To detect animal name via image

**2. Data preprocessing (No augmentation)**

In [32]:
image_size = (224, 224)
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    "animal_data_file/animal_data",
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="training",
    shuffle=True
)

val_generator = datagen.flow_from_directory(
    "animal_data_file/animal_data",
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation",
    shuffle=False
)

num_classes = train_generator.num_classes

Found 1561 images belonging to 15 classes.
Found 383 images belonging to 15 classes.


**3. Hypermodel using MobileNet**

In [34]:
def build_cnn_model(hp):
    base_model = MobileNet(include_top=False, weights='imagenet', input_shape=image_size + (3,))
    base_model.trainable = False  # Freeze base

    model = Sequential()
    model.add(base_model)
    model.add(GlobalAveragePooling2D())

    units = hp.Int("units", min_value=64, max_value=256, step=32)
    model.add(Dense(
        units=units,
        activation=hp.Choice("activation", ['relu', 'tanh']),
        kernel_initializer=hp.Choice("initializer", ['glorot_uniform', 'he_normal']),
        kernel_regularizer=regularizers.l2(
            hp.Float("lambda", 1e-4, 1e-2, sampling='log'))
    ))

    model.add(Dropout(hp.Float("dropout", 0.2, 0.5, step=0.1)))
    model.add(Dense(num_classes, activation='softmax'))

    learning_rate = hp.Float("learning_rate", 1e-4, 1e-2, sampling='log')
    optimizer_choice = hp.Choice("optimizer", ['adam', 'sgd', 'rmsprop'])

    if optimizer_choice == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate)
    elif optimizer_choice == 'sgd':
        optimizer = tf.keras.optimizers.SGD(learning_rate)
    else:
        optimizer = tf.keras.optimizers.RMSprop(learning_rate)

    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model


**4. Keras Tuner Setup**

In [35]:
tuner = kt.RandomSearch(
    build_cnn_model,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    directory='cnn_tuning',
    project_name='mobilenet_model'
)

early_stop = EarlyStopping(monitor='val_loss', patience=5)

**5. Start tuning**

In [36]:
tuner.search(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=[early_stop],
    verbose=2
)

Trial 10 Complete [00h 00m 48s]
val_loss: 0.39432650804519653

Best val_loss So Far: 0.20354406535625458
Total elapsed time: 00h 08m 30s


**6. Save best model**

In [37]:
best_model = tuner.get_best_models(1)[0]
best_hyperparams = tuner.get_best_hyperparameters(1)[0]

print("\nBest Hyperparameters:")
print(best_hyperparams.values)

if not os.path.exists("model"):
    os.mkdir("model")

best_model.save("model/mobilenet_best_tuned_model.h5")

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))



Best Hyperparameters:
{'units': 64, 'activation': 'relu', 'initializer': 'glorot_uniform', 'lambda': 0.0002136185900215609, 'dropout': 0.4, 'learning_rate': 0.002983193841558442, 'optimizer': 'rmsprop'}


**Evaluation**

In [38]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import numpy as np

# Predict the class probabilities
y_pred_train_prob = best_model.predict(train_generator)

# Get the predicted class index (highest probability)
y_pred_train = np.argmax(y_pred_train_prob, axis=1)

# Get the true class labels
y_train = train_generator.classes

# Print evaluation metrics
print("Confusion Matrix:")
print(confusion_matrix(y_train, y_pred_train))

print("\nClassification Report:")
class_labels = list(train_generator.class_indices.keys())  # Get class names
print(classification_report(y_train, y_pred_train, target_names=class_labels))


49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 88ms/step
Confusion Matrix:
[[ 5  8  6  7  6  4  2  6 11 11  4  6 11  8  5]
 [ 7  4  6  8  9  9  4 11 10  9  8  6  4 10  5]
 [13  7  5  7  6 10  8  6  8  5  5  4  5  5  5]
 [ 3 11  4  8  5  4  8  7  9 10  7  6  6  6 11]
 [ 8  4  4  6 10  2  8  7  7  4 10  9  5 10  8]
 [10  8  7  7  5  4  4  7  4  9  5  7  7  9  5]
 [ 3  6  7 10  8 10  8 11  4  3  5  5  7  9  8]
 [ 5 11  2  4  8  9  8  7  6  7  7  7  8  5 13]
 [ 5 11  8  6  8  7  3  7  6 10  7  6  4  8  8]
 [ 7  8  2  4  5 14  6  9 10  7  7  7  8  4  6]
 [ 3  6  9  5  6  3  8  7  5  6  8  6 12  7 10]
 [10  8  6  8 11  5 12  6  4  4  8  8  7  1  7]
 [ 4  9  9  9  4  9  6  5  5  6  4 10 11 11  6]
 [10  6  6  6 11  5  8  6  9  7  8  4  5  4  9]
 [ 8  5 14  9  4  5 11  5  5  7  6 12  8  7  4]]

Classification Report:
              precision    recall  f1-score   support

        Bear       0.05      0.05      0.05       100
        Bird       0.04      0.04      0.04       110
         Cat       0.05      0.05 

In [40]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    'animal_data_file/animal_data',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


Found 1944 images belonging to 15 classes.


In [41]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import numpy as np

# Step 1: Predict on test data
y_pred_test_prob = best_model.predict(test_generator)

# Step 2: Get the predicted class index (highest probability)
y_pred_test = np.argmax(y_pred_test_prob, axis=1)

# Step 3: Get the true class labels from the test generator
y_test = test_generator.classes

# Step 4: Class labels (same as training)
class_labels = list(test_generator.class_indices.keys())

# Step 5: Print metrics
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_test))

print("\nClassification Report:")
print(classification_report(y_test, y_pred_test, target_names=class_labels))


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


61/61 ━━━━━━━━━━━━━━━━━━━━ 4s 72ms/step
Confusion Matrix:
[[125   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0 137   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   2 118   0   1   2   0   0   0   0   0   0   0   0   0]
 [  0   0   0 130   0   1   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0 127   0   0   0   0   0   0   0   0   0   0]
 [  1   0   0   0   0 121   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0 129   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   1   1 131   0   0   0   0   0   0   0]
 [  0   0   0   0   1   0   0   0 128   0   0   0   0   0   0]
 [  1   0   0   0   1   1   0   1   0 126   0   0   0   0   0]
 [  0   0   0   1   5   0   0   0   0   0 120   0   0   0   0]
 [  1   0   0   1   0   1   0   1   0   1   0 126   0   0   0]
 [  0   0   0   0   0   1   0   0   0   0   0   0 134   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0 129   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0 13

In [43]:
print(train_generator.class_indices)


{'Bear': 0, 'Bird': 1, 'Cat': 2, 'Cow': 3, 'Deer': 4, 'Dog': 5, 'Dolphin': 6, 'Elephant': 7, 'Giraffe': 8, 'Horse': 9, 'Kangaroo': 10, 'Lion': 11, 'Panda': 12, 'Tiger': 13, 'Zebra': 14}


In [44]:
train_generator = datagen.flow_from_directory(
    "animal_data_file/animal_data",          # NOT the folder with all images directly inside
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True
)



Found 1561 images belonging to 15 classes.


In [46]:
val_prob = best_model.predict(val_generator)
val_pred = np.argmax(val_prob, axis=1)
val_true = val_generator.classes

from sklearn.metrics import classification_report
print(classification_report(val_true, val_pred, target_names=class_labels))


12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 291ms/step
              precision    recall  f1-score   support

        Bear       0.93      1.00      0.96        25
        Bird       1.00      1.00      1.00        27
         Cat       1.00      0.96      0.98        24
         Cow       0.93      1.00      0.96        26
        Deer       0.86      1.00      0.93        25
         Dog       0.82      0.96      0.88        24
     Dolphin       0.96      1.00      0.98        25
    Elephant       0.92      0.92      0.92        26
     Giraffe       1.00      1.00      1.00        25
       Horse       1.00      0.85      0.92        26
    Kangaroo       1.00      0.84      0.91        25
        Lion       1.00      0.88      0.94        26
       Panda       1.00      0.96      0.98        27
       Tiger       1.00      1.00      1.00        25
       Zebra       1.00      1.00      1.00        27

    accuracy                           0.96       383
   macro avg       0.96      0.96      

### Evaluate Accuracy on Validation/Test Data

In [53]:
val_loss, val_accuracy = best_model.evaluate(val_generator, verbose=1)

print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")
print(f"Validation Loss: {val_loss:.4f}")


12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 146ms/step - accuracy: 0.9742 - loss: 0.1646
Validation Accuracy: 95.82%
Validation Loss: 0.2035
